##### APPROACH:

Further fine-tuning an already fine-tuned model(HATE-SPEECH-BERT) for multilabel classification to identify subcategories of hate speech requires us to assess whether there are improvements in accuracy, precision, recall (F1 score) by initially using a triple classification model for hate speech detection before fine-tuning for multilabel classification. Alternatively, it may be worthwhile to directly fine-tune the model(BERT-LARGE) for multilabel classification from the start

#### Import necessary libraries

In [1]:
! pip install contractions
! pip install -U accelerate
! pip install -U transformers

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 880, in try_parse
    return self._parse(instring, loc, doActions=False)[0]
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 821, in _parseNoCache
    loc, tokens = self.parseImpl(instring, pre_loc, doActions)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 2987, in parseImpl
    raise ParseException(instring, loc, self.errmsg, self)
pi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import numpy as np
import re
import contractions
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from collections import defaultdict
import time
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


#### Loading the pretrained_model
##### [hateBert](https://huggingface.co/GroNLP/hateBERT) Already fine-tuned model

In [2]:
## get the model from the huggingface model hub
!git clone https://huggingface.co/GroNLP/hateBERT

fatal: destination path 'hateBERT' already exists and is not an empty directory.


In [3]:
## If you download the model locally , you can load it from the local path
PATH = os.getcwd()+"/hateBERT"

## Load the BERT model
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained(PATH)
model = BertModel.from_pretrained(PATH)

print(model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [4]:
# Load the model weights using hugingface model hub
from transformers import BertTokenizer, BertModel
import torch

# Example of getting the output of the model for a given text
def tokenize_text(text):
    return tokenizer(text, padding=True, truncation=True, return_tensors="pt")

# Use the model in inference mode and classify a give example
def classify(text):
    inputs = tokenize_text(text)
    print(inputs)
    outputs = model(**inputs)
    return outputs

text = "Hello World"
outputs = classify(text)
print(outputs)

{'input_ids': tensor([[ 101, 7592, 2088,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.6163,  0.4899,  0.6045,  ..., -0.0070,  0.6881,  0.5794],
         [-0.7543, -0.0851,  0.9398,  ..., -0.3023, -0.1036,  0.7563],
         [-0.4167,  0.3670,  0.5797,  ..., -0.2335,  0.2327, -0.3771],
         [ 0.1706,  0.1823,  1.2238,  ...,  0.1885,  1.3003,  0.2482]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8061, -0.1458,  0.9448,  0.6965, -0.7556, -0.2725,  0.7985,  0.1897,
          0.8852, -0.9938,  0.4638, -0.3088,  0.9876, -0.8221,  0.9338, -0.0994,
         -0.0384, -0.3089,  0.4715, -0.3408,  0.7167, -0.6686,  0.7346, -0.0056,
          0.0972, -0.5668, -0.4905,  0.9583,  0.9329,  0.7648, -0.6556,  0.1152,
         -0.9940, -0.3204,  0.8402, -0.9773, -0.0690, -0.6670, -0.0889, -0.0813,
         -0.9264,  0.3244,  0.9340,  0.2341,  0.0877, 

##### Import the multilabel data
##### [UCBerkeley - Hate Speech Dataset (Multilabel)](https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech)


In [5]:
# Download the data locally
# https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech/tree/main

## Read the measuring-hate-speech.parquet
parquet_data = pd.read_parquet('measuring-hate-speech.parquet')

## remove from df the redundant  columns
multilabel_dataset = parquet_data.iloc[:, :-116]

## Remove comment_idm,annotator_id,platform and put the column text in the first position
multilabel_dataset = multilabel_dataset[['text','sentiment', 'hatespeech', 'hate_speech_score', 'respect','insult','humiliate','status','dehumanize','violence','genocide' ,'attack_defend']]
multilabel_dataset

,text,sentiment,hatespeech,hate_speech_score,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend
0,Yes indeed. She sort of reminds me of the elde...,0.0,0.0,-3.90,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,The trans women reading this tweet right now i...,0.0,0.0,-6.52,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0
2,Question: These 4 broads who criticize America...,4.0,2.0,0.36,4.0,4.0,4.0,4.0,4.0,0.0,0.0,4.0
3,It is about time for all illegals to go back t...,2.0,0.0,0.26,3.0,2.0,1.0,2.0,0.0,0.0,0.0,3.0
4,For starters bend over the one in pink and kic...,4.0,2.0,1.54,4.0,4.0,4.0,4.0,4.0,4.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
135551,عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي نص...,1.0,0.0,-4.88,1.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0
135552,Millions of #Yemen-is participated in mass ral...,2.0,0.0,-4.40,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
135553,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,1.0,0.0,-2.49,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
135554,Millions of #Yemen-is participated in mass ral...,2.0,0.0,-4.40,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0


### Preprocessing text data

In [6]:
## Preprocess the TEXT data
## Remove HTML tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

multilabel_dataset['text'] = multilabel_dataset['text'].apply(lambda x: remove_html_tags(x))

## Remove URL
def remove_url(text):
    return re.sub(r'http\S+', '', text)

multilabel_dataset['text'] = multilabel_dataset['text'].apply(lambda x: remove_url(x))

## Lowercase
multilabel_dataset['text'] = multilabel_dataset['text'].str.lower()

## Remove punctuation
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

multilabel_dataset['text'] = multilabel_dataset['text'].apply(lambda x: remove_punctuation(x))

## Handling Contractions using libraries

def expand_contractions(text):
    return contractions.fix(text)

multilabel_dataset['text'] = multilabel_dataset['text'].apply(lambda x: expand_contractions(x))

## Remove stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

multilabel_dataset['text'] = multilabel_dataset['text'].apply(lambda x: remove_stopwords(x))

## Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
multilabel_dataset['text'] = multilabel_dataset['text'].apply(lambda x: lemmatize_words(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Preprocess labeling

In [7]:
## Replace each catefory of hatespeech with 0 or 1 for the columns respect, insult, humiliate, status, dehumanize, violence, genocide, attack_defend
## Do it more efficiently

# List of columns to transform
columns_to_transform = ['respect', 'insult', 'humiliate', 'status',
                        'dehumanize', 'violence', 'genocide', 'attack_defend']

# Apply the transformation
for column in columns_to_transform:
    multilabel_dataset[column] = multilabel_dataset[column].apply(lambda x: 1 if x > 0 else 0)
multilabel_dataset


,text,sentiment,hatespeech,hate_speech_score,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend
0,yes indeed sort reminds elder lady played part...,0.0,0.0,-3.90,0,0,0,1,0,0,0,0
1,trans woman reading tweet right beautiful,0.0,0.0,-6.52,0,0,0,1,0,0,0,1
2,question 4 broad criticize america country fle...,4.0,2.0,0.36,1,1,1,1,1,0,0,1
3,time illegals go back country origin keep free...,2.0,0.0,0.26,1,1,1,1,0,0,0,1
4,starter bend one pink kick as pussy get taste ...,4.0,2.0,1.54,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
135551,عاجل سماحة السيد_عبدالملك_بدرالدين_الحوثي نصره...,1.0,0.0,-4.88,1,0,0,1,0,0,0,1
135552,million yemeni participated mass rally 13squar...,2.0,0.0,-4.40,0,0,0,1,0,0,0,1
135553,abeshinzo realdonaldtrump shinzoabe 独裁者は行きますこれ...,1.0,0.0,-2.49,1,1,1,1,0,0,0,1
135554,million yemeni participated mass rally 13squar...,2.0,0.0,-4.40,0,0,0,1,0,0,0,1


In [8]:
multilabel_dataset['labels']=multilabel_dataset[columns_to_transform].values.tolist()
multilabel_dataset = multilabel_dataset[multilabel_dataset['text'] != '']
multilabel_dataset

,text,sentiment,hatespeech,hate_speech_score,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend,labels
0,yes indeed sort reminds elder lady played part...,0.0,0.0,-3.90,0,0,0,1,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0]"
1,trans woman reading tweet right beautiful,0.0,0.0,-6.52,0,0,0,1,0,0,0,1,"[0, 0, 0, 1, 0, 0, 0, 1]"
2,question 4 broad criticize america country fle...,4.0,2.0,0.36,1,1,1,1,1,0,0,1,"[1, 1, 1, 1, 1, 0, 0, 1]"
3,time illegals go back country origin keep free...,2.0,0.0,0.26,1,1,1,1,0,0,0,1,"[1, 1, 1, 1, 0, 0, 0, 1]"
4,starter bend one pink kick as pussy get taste ...,4.0,2.0,1.54,1,1,1,1,1,1,1,1,"[1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,عاجل سماحة السيد_عبدالملك_بدرالدين_الحوثي نصره...,1.0,0.0,-4.88,1,0,0,1,0,0,0,1,"[1, 0, 0, 1, 0, 0, 0, 1]"
135552,million yemeni participated mass rally 13squar...,2.0,0.0,-4.40,0,0,0,1,0,0,0,1,"[0, 0, 0, 1, 0, 0, 0, 1]"
135553,abeshinzo realdonaldtrump shinzoabe 独裁者は行きますこれ...,1.0,0.0,-2.49,1,1,1,1,0,0,0,1,"[1, 1, 1, 1, 0, 0, 0, 1]"
135554,million yemeni participated mass rally 13squar...,2.0,0.0,-4.40,0,0,0,1,0,0,0,1,"[0, 0, 0, 1, 0, 0, 0, 1]"


#### Dataset/Dataloader

In [9]:
#X = multilabel_dataset['text'].values.tolist()
#encoded_inputs = tokenizer(X, padding=True, truncation=True,max_length=256, return_tensors='pt')

In [10]:
#labels = multilabel_dataset[columns_to_transform].values  # Assuming you have extracted binary labels
#encoded_inputs['labels']=torch.tensor(labels, dtype=torch.float)

In [11]:
#encoded_inputs

In [12]:
#inputs = multilabel_dataset['text'].values
#labels = multilabel_dataset[columns_to_transform].values

In [13]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [15]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, data, tokenizer,max_len=256):

        self.tokenizer = tokenizer
        self.data = data
        self.text = data.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        text = str(self.text.iloc[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )


        ids = encoding['input_ids']
        mask = encoding['attention_mask']
        token_type_ids = encoding['token_type_ids']

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets.iloc[idx], dtype=torch.float).to(device)
        }


In [16]:
train_texts, val_texts,= train_test_split(multilabel_dataset,test_size=0.2)

print("Dataset length: {}".format(multilabel_dataset.shape))
print("Train Dataset length: {}".format(train_texts.shape))
print("Val Dataset length: {}".format(val_texts.shape))

Dataset length: (135555, 13)
Train Dataset length: (108444, 13)
Val Dataset length: (27111, 13)


In [17]:
training_dataset = HateSpeechDataset(train_texts,tokenizer,MAX_LEN)
validation_dataset = HateSpeechDataset(val_texts,tokenizer,MAX_LEN)

In [18]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_dataset, **train_params)
validation_loader = DataLoader(validation_dataset, **test_params)

In [19]:
next(iter(training_loader))

{'input_ids': tensor([[  101,  7564,  3350,  ...,     0,     0,     0],
         [  101,  2729,  5561,  ...,     0,     0,     0],
         [  101, 20342,  2004,  ...,     0,     0,     0],
         ...,
         [  101,  2052, 24357,  ...,     0,     0,     0],
         [  101,  2304,  2796,  ...,     0,     0,     0],
         [  101,  4297,  9050,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'targets': tensor([[1., 0., 0., 1., 1., 0., 0., 1.],
         [1., 1., 1., 1., 1., 0., 0., 1.],
         [1., 1., 1., 1., 1., 0.

#### Fine Tuning the model

In [20]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [21]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class MultilabelHateBert(torch.nn.Module):
    def __init__(self,bertmodel):
        super(MultilabelHateBert, self).__init__()
        self.bertmodel = bertmodel
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, len(columns_to_transform))

    def forward(self, ids, mask, token_type_ids):

      output_1= self.bertmodel(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output_2 = self.dropout(output_1.pooler_output)
      output = self.linear(output_2)
      return output

multilabel_model = MultilabelHateBert(model)
multilabel_model.to(device)

MultilabelHateBert(
  (bertmodel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [22]:
def criterion(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [23]:
optimizer = torch.optim.Adam(params =  multilabel_model.parameters(), lr=LEARNING_RATE)


#### Training Loop

In [ ]:
from tqdm import tqdm
for epoch in tqdm(range(EPOCHS), desc="Epochs"):

  multilabel_model.train()

  for i, batch in tqdm(enumerate(training_loader), desc=f"Epoch {epoch + 1}", total=len(training_loader)):

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    targets = batch['targets'].to(device)

    outputs = multilabel_model(input_ids,attention_mask,token_type_ids)
    optimizer.zero_grad()
    loss = criterion(outputs,targets)

    if i%50==0:
      print()
      print(f'Epoch: {epoch}, Loss{loss.item()}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch 1:   0%|          | 0/6778 [00:00<?, ?it/s]


Epoch: 0, Loss0.6766090393066406



Epoch 1:   0%|          | 21/6778 [00:13<1:15:18,  1.50it/s]

#### Validation Loop

In [ ]:
from sklearn import metrics

for epoch in tqdm(range(EPOCHS)):

  multilabel_model.eval()
  val_targets=[]
  val_outputs=[]

  with torch.no_grad():

    for i,batch in enumerate(validation_loader):

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      token_type_ids = batch['token_type_ids'].to(device)
      targets = batch['targets'].to(device)

      outputs = multilabel_model(input_ids,attention_mask,token_type_ids)

      val_targets.extend(targets.cpu().detach().numpy().tolist())
      val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

  outputs = np.array(val_outputs) >= 0.5
  accuracy = metrics.accuracy_score(val_targets, outputs)
  f1_score_micro = metrics.f1_score(val_targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(val_targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
multilabel_model

In [ ]:
torch.save(multilabel_model.state_dict(), '/content/drive/fine_tuning_v1.2')

In [ ]:

#multilabel_model1 = MultilabelHateBert(model)
#multilabel_model1.load_state_dict(torch.load('/content/fine_tuning_v1.1'))

## Interpretability

In [ ]:
multilabel_dataset

In [ ]:
#Import
! pip install lime
! pip install transformers-interpret

In [ ]:
samples = [17, 4, 44, 3, 8, 27, 71, 74]
attributions = dict()
attributions['LIME'] = []

In [ ]:
import lime
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=columns_to_transform, split_expression='\s+', bow=False)

In [ ]:

def predictor(texts):

    encodings = tokenizer(texts,
                           padding='max_length',
                           truncation=True,
                           max_length=MAX_LEN,
                           return_tensors='pt')

    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    token_type_ids = encodings['token_type_ids'].to(device)

    logits = multilabel_model(input_ids, attention_mask, token_type_ids)
    probabilities = F.softmax(logits, dim=1)

    return probabilities.cpu().detach().numpy()

for idx in samples:
    instance = val_texts.iloc[idx].text

    # Call predictor with the current instance
    exp = explainer.explain_instance(instance, predictor, num_features=200, num_samples=64)

    explanation_dict = dict(list(exp.as_map().values())[0])
    tokens = val_texts.iloc[idx].text.split(' ')
    scores = []

    for i in range(len(tokens)):
        scores.append((tokens[i], explanation_dict[i]))

    attributions['LIME'].append(scores)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm, transforms

# Plotting Code from innvestigate library: https://github.com/albermax/innvestigate
def plot_text_heatmap(words, scores, title="", width=10, height=0.2, verbose=0, max_word_per_line=20):
    fig = plt.figure(figsize=(width, height))

    ax = plt.gca()

    ax.set_title(title, loc='left')
    tokens = words
    if verbose > 0:
        print('len words : %d | len scores : %d' % (len(words), len(scores)))

    cmap = plt.cm.ScalarMappable(cmap=cm.bwr)
    cmap.set_clim(0, 1)

    canvas = ax.figure.canvas
    t = ax.transData

    # normalize scores to the followings:
    # - negative scores in [0, 0.5]
    # - positive scores in (0.5, 1]
    normalized_scores = 0.5 * scores / np.max(np.abs(scores)) + 0.5

    if verbose > 1:
        print('Raw score')
        print(scores)
        print('Normalized score')
        print(normalized_scores)

    # make sure the heatmap doesn't overlap with the title
    loc_y = -0.2

    for i, token in enumerate(tokens):
        *rgb, _ = cmap.to_rgba(normalized_scores[i], bytes=True)
        color = '#%02x%02x%02x' % tuple(rgb)

        text = ax.text(0.0, loc_y, token,
                       bbox={
                           'facecolor': color,
                           'pad': 5.0,
                           'linewidth': 1,
                           'boxstyle': 'round,pad=0.5'
                       }, transform=t)

        text.draw(canvas.get_renderer())
        ex = text.get_window_extent()

        # create a new line if the line exceeds the length
        if (i+1) % max_word_per_line == 0:
            loc_y = loc_y -  2.5
            t = ax.transData
        else:
            t = transforms.offset_copy(text._transform, x=ex.width+15, units='dots')

    if verbose == 0:
        ax.axis('off')

In [ ]:
# Plotting
methods = ['LIME']

for sample_id in range(len(samples)):
    for method in methods:
        analysis = attributions[method][sample_id]
        words = [t[0] for t in analysis]
        scores = np.array([t[1] for t in analysis])
        plot_text_heatmap(words, scores, title='Method: %s' % method, verbose=0)
    plt.show()